<a href="https://colab.research.google.com/github/swamyputta/Sales-Prediction/blob/main/MovieReview_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install kaggle


**Importing
 the Dependencies**

In [33]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding ,LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


**Data Collection -Kaggle API**


In [34]:
kaggle_dictionary = json.load(open("kaggle.json"))

In [35]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [36]:
#setup kaggle credentials as environment  variables
os.environ["KAGGLE_USERNAME"]=kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"]=kaggle_dictionary["key"]

In [37]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [38]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


In [39]:
#unzip the dataset file
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip","r") as zip_ref:
  zip_ref.extractall()

In [40]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


**Loading the dataset**

In [41]:
data=pd.read_csv("/content/IMDB Dataset.csv")

In [42]:
data.shape

(50000, 2)

In [43]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [44]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [45]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [46]:
data.replace({"sentiment":{"positive":1,"negative":0}},inplace=True)

<ipython-input-46-329ec005e21e>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment":{"positive":1,"negative":0}},inplace=True)


In [47]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [48]:
data["sentiment"].value_counts()

,count
sentiment,
1,25000
0,25000


In [49]:
#splitting the dataset into training data and test data
train_data, test_data = train_test_split(data,test_size=0.2,random_state=42)

In [50]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


**Data Preprocessing**

In [62]:
#tokenize the text data
tokenizer=Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
x_train=pad_sequences(tokenizer.texts_to_sequences(train_data["review"]),maxlen=200)
x_test=pad_sequences(tokenizer.texts_to_sequences(test_data["review"]),maxlen=200)

In [63]:
print(x_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [64]:
print(x_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [65]:
y_train=train_data["sentiment"]
y_test=test_data["sentiment"]

In [66]:
print(y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


**Building LSTM (Long short-Term Memory)**

In [67]:
#building the model

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128 , input_length=200))
model.add(LSTM(128, dropout=0.2 , recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

In [69]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [70]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [71]:
import numpy as np
x = np.random.randint(0, 5000, size=(1, 200))
model.build(input_shape=(None, 200))

In [72]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ (None, 200, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [74]:
 # compile the model
 model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

**Training our model**

In [75]:
model.fit(x_train , y_train, epochs=5,batch_size=64 , validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 156s 299ms/step - accuracy: 0.7232 - loss: 0.5290 - val_accuracy: 0.8447 - val_loss: 0.3616
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 198s 303ms/step - accuracy: 0.8541 - loss: 0.3526 - val_accuracy: 0.8501 - val_loss: 0.3469
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 185s 268ms/step - accuracy: 0.8686 - loss: 0.3120 - val_accuracy: 0.8646 - val_loss: 0.3465
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 137s 258ms/step - accuracy: 0.8788 - loss: 0.2995 - val_accuracy: 0.8612 - val_loss: 0.3484
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 166s 308ms/step - accuracy: 0.8918 - loss: 0.2683 - val_accuracy: 0.8637 - val_loss: 0.3446


**Model Evaluation**

In [76]:
loss , accuracy= model.evaluate(x_test,y_test)
print(f"Test loss : {loss}")
print(f"Test Accuracy : {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 29s 92ms/step - accuracy: 0.8705 - loss: 0.3240
Test loss : 0.32350772619247437
Test Accuracy : 0.8720999956130981


**Building a Predictive System**

In [92]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences(review)
  padded_sequence = pad_sequences(sequence ,maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [93]:
# exxample usage

new_review = "This movie is terrible. I literally hate it."
sentiment = predict_sentiment([new_review])
print(f"The sentiment of the review is :{sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
The sentiment of the review is :negative


In [94]:
new_review = "This movie is awesome. I really like it."
sentiment = predict_sentiment([new_review])
print(f"The sentiment of the review is :{sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
The sentiment of the review is :positive


In [96]:
new_review = "This movie is ok but not that good."
sentiment = predict_sentiment([new_review])
print(f"The sentiment of the review is :{sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
The sentiment of the review is :negative
